Based on: https://datascience.quantecon.org/applications/maps.html <br>

In [1]:
from pathlib import Path
from shapely.geometry import Point
import os
import sys
import csv
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

ModuleNotFoundError: No module named 'geopandas'

In [2]:
# Basic paths
project_path = Path().resolve().parent
csv_path = project_path / "speeches_csv"

In [3]:
# Open in pandas the table with the sentiment and the location information
df_sen = pd.read_csv("results_sa_all.txt")
df_sen.head()

,title,date,stanza,textblob,vader,subjectivity
0,Prayer_Breakfast_2016,2016-02-04,0.099,0.178,1.000,0.542
1,Security_Team_Announcement,2008-12-01,0.140,0.162,0.998,0.403
2,Cairo_University,2009-06-04,-0.150,0.110,1.000,0.457
3,Umpqua_Community_College_Shootings,2015-10-01,-0.250,0.061,-0.998,0.435
4,White_House_Correspondent_Dinner_2013,2013-04-27,-0.012,0.145,1.000,0.501
...,...,...,...,...,...,...
283,Shimon_Peres_Memorial,2016-09-30,-0.083,0.164,1.000,0.436
284,ASEAN_Business_2015,2015-11-21,-0.050,0.164,1.000,0.399
285,Finance_Crisis_Fee,2010-01-14,-0.412,0.080,0.987,0.436
286,Oval_Office_Counter_Terrorism_Agenda,2015-12-06,-0.134,0.076,-1.000,0.398


In [20]:
# Call the all_speechs dataframe and clean it a bit
df_temp = pd.read_csv(csv_path / "all_speeches_cleaned.txt") 
df_temp = df_temp.drop(columns=["location","count_commas","content","date"])
df_temp = df_temp[ df_temp["highest_speaker_count"] <= 3 ]
df_temp.head()

,title,pages,highest_speaker_count,country,state,city,specific_location
1,Prayer_Breakfast_2016,7,0,USA,no_state,Washington D.C.,Washington Hilton
2,Security_Team_Announcement,5,0,USA,Illinois,Chicago,no_specific_location
3,Cairo_University,14,0,Egypt,no_state,Cairo,no_specific_location
4,Umpqua_Community_College_Shootings,4,0,USA,no_state,Washington D.C.,no_specific_location
5,White_House_Correspondent_Dinner_2013,6,0,USA,no_state,Washington D.C.,Washington Hilton Hotel


In [21]:
# Merge the previous tables
df_cb = pd.merge(df_temp, df_sen, how='inner', on = 'title')
df_cb.head()

,title,pages,highest_speaker_count,country,state,city,specific_location,date,stanza,textblob,vader,subjectivity
0,Prayer_Breakfast_2016,7,0,USA,no_state,Washington D.C.,Washington Hilton,2016-02-04,0.099,0.178,1.000,0.542
1,Security_Team_Announcement,5,0,USA,Illinois,Chicago,no_specific_location,2008-12-01,0.140,0.162,0.998,0.403
2,Cairo_University,14,0,Egypt,no_state,Cairo,no_specific_location,2009-06-04,-0.150,0.110,1.000,0.457
3,Umpqua_Community_College_Shootings,4,0,USA,no_state,Washington D.C.,no_specific_location,2015-10-01,-0.250,0.061,-0.998,0.435
4,White_House_Correspondent_Dinner_2013,6,0,USA,no_state,Washington D.C.,Washington Hilton Hotel,2013-04-27,-0.012,0.145,1.000,0.501
